In [225]:
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import random


## Merge data characteristics and regression results.

In [226]:
df_regression = pd.read_excel("../stats/final_table.xlsx")
bx_age = pd.read_csv("../stats/stats_BX_age.csv")
ml1m_age = pd.read_csv("../stats/stats_ml1m_age.csv")
ml1m_gender = pd.read_csv("../stats/stats_ml1m_gender.csv")

bx_age["Sensitive Feature"] = "Age"
ml1m_age["Sensitive Feature"] = "Age"
ml1m_gender["Sensitive Feature"] = "Gender"

df_concat = pd.concat([bx_age, ml1m_age, ml1m_gender], axis=0, ignore_index=True)

In [227]:
"""
# Compute the correlation matrix
correlation_matrix = df_regression[['ndcg@5', 'recall@5', 'mrr@5',
       'Differential Fairness of sensitive attribute', 'giniindex@5',
       'popularitypercentage@5', 'Value Unfairness of sensitive attribute',
       'Absolute Unfairness of sensitive attribute',
       'Underestimation Unfairness of sensitive attribute',
       'Overestimation Unfairness of sensitive attribute',
       'NonParity Unfairness of sensitive attribute', 'Absolute Difference',
       'KS Statistic of sensitive attribute', 'Generalized Cross Entropy',
       'hit@5']].corr()

# Plot the heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Feature Correlation Heatmap")
plt.show()"""

'\n# Compute the correlation matrix\ncorrelation_matrix = df_regression[[\'ndcg@5\', \'recall@5\', \'mrr@5\',\n       \'Differential Fairness of sensitive attribute\', \'giniindex@5\',\n       \'popularitypercentage@5\', \'Value Unfairness of sensitive attribute\',\n       \'Absolute Unfairness of sensitive attribute\',\n       \'Underestimation Unfairness of sensitive attribute\',\n       \'Overestimation Unfairness of sensitive attribute\',\n       \'NonParity Unfairness of sensitive attribute\', \'Absolute Difference\',\n       \'KS Statistic of sensitive attribute\', \'Generalized Cross Entropy\',\n       \'hit@5\']].corr()\n\n# Plot the heatmap\nplt.figure(figsize=(8, 6))\nsns.heatmap(correlation_matrix, annot=True, cmap=\'coolwarm\', fmt=".2f", linewidths=0.5)\nplt.title("Feature Correlation Heatmap")\nplt.show()'

In [228]:
# Fill NaN for "Value Unfairness" using group mean
df_regression["Value Unfairness of sensitive attribute"] = df_regression.groupby(["Model Name", "Sensitive Feature"])[
    "Value Unfairness of sensitive attribute"].transform(
    lambda x: x.fillna(x.mean())
)

# Fill NaN for "Absolute Unfairness" using group median
df_regression["Absolute Unfairness of sensitive attribute"] = df_regression.groupby(["Model Name", "Sensitive Feature"])[
    "Absolute Unfairness of sensitive attribute"].transform(
    lambda x: x.fillna(x.mean())
)

# Fill NaN for "Underestimation Unfairness" using group mean
df_regression["Underestimation Unfairness of sensitive attribute"] = df_regression.groupby(["Model Name", "Sensitive Feature"])[
    "Underestimation Unfairness of sensitive attribute"].transform(
    lambda x: x.fillna(x.mean()))

In [229]:
df_regression.replace([np.inf, -np.inf], np.nan, inplace=True)

df_final = df_regression.merge(
    df_concat, 
    how="left", 
    on=['Dataset', 'Subset ID', 'Is Filtered', 'Sensitive Feature']
)

In [230]:

# Function to replace NaN values in all numeric columns with group-specific mean + optional noise
def replace_nan_with_group_mean_all(df, groupby_cols, add_noise=False):
    numeric_cols = df.select_dtypes(include=[np.number]).columns  # Select numeric columns
    
    for col in numeric_cols:
        # Compute group-specific mean and std for the current column
        group_stats = df.groupby(groupby_cols)[col].agg(['mean', 'std'])
        
        # Function to replace NaN values
        def replace_nan(row, col, group_stats, add_noise):
            if pd.isna(row[col]):
                mean_value = group_stats.loc[(row[groupby_cols[0]], row[groupby_cols[1]], row[groupby_cols[2]]), 'mean']
                if add_noise:
                    std_value = group_stats.loc[(row[groupby_cols[0]], row[groupby_cols[1]], row[groupby_cols[2]]), 'std']
                    noise = random.uniform(-0.5 * std_value, 0.5 * std_value) if pd.notna(std_value) else 0
                    return mean_value + noise
                return mean_value
            return row[col]
        
        # Apply the replacement function to the column
        df[col] = df.apply(replace_nan, axis=1, col=col, group_stats=group_stats, add_noise=add_noise)
    
    return df

In [231]:
groupby_columns = ["Model Name", "Dataset", "Sensitive Feature"]
df_final = replace_nan_with_group_mean_all(df_final, groupby_columns, add_noise=True)

In [232]:
df_final.shape

(732, 45)

In [126]:
df_final.to_csv("df_regression.csv", index=False)